# 案例: 使用LSTM进行流量预测

本案例演示了如何使用 `LSTMFlowForecaster` 智能体。LSTM（长短期记忆网络）是一种强大的深度学习模型，尤其擅长处理时间序列数据中的复杂模式。

### 1. 导入库并加载配置

In [ ]:
import json
import numpy as np
import matplotlib.pyplot as plt

from swp.local_agents.prediction.lstm_forecaster import LSTMFlowForecaster
from swp.central_coordination.collaboration.message_bus import MessageBus

CONFIG_PATH = 'examples/lstm_forecasting.json'
with open(CONFIG_PATH, 'r') as f:
    config = json.load(f)

print("配置加载成功！")

# 设置 Matplotlib 样式
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

### 2. 生成合成数据

我们根据配置生成一个包含正弦波、上升趋势和一些随机噪声的合成时间序列，用于训练和测试模型。

In [ ]:
def generate_data(cfg):
    data_cfg = cfg['data_generation']
    time_steps = np.linspace(0, data_cfg['time_steps_end'], data_cfg['total_points'])
    data = (
        data_cfg['sin_wave_multiplier'] * np.sin(time_steps) + 
        data_cfg['trend_multiplier'] * time_steps**2 + 
        np.random.randn(data_cfg['total_points']) * data_cfg['noise_multiplier'] + 
        data_cfg['base_value']
    )
    print(f"成功生成 {data_cfg['total_points']} 个合成数据点。")
    return data

data = generate_data(config)

### 3. 训练模型并生成预测

In [ ]:
# 1. 设置智能体
bus = MessageBus()
agent_cfg = config['agent_config']
lstm_agent = LSTMFlowForecaster("lstm_forecaster_1", bus, agent_cfg)

# 2. 向智能体提供历史数据
training_data = data[:agent_cfg["history_size"]]
for value in training_data:
    lstm_agent.handle_observation_message({'value': value}) # 直接调用handler以简化

# 3. 运行智能体以训练模型
print("\n--- 开始训练 LSTM 模型 ---")
lstm_agent.run(current_time=1)
print("--- 训练完成 ---")

# 4. 生成预测
print("\n--- 开始生成预测 ---")
forecast = lstm_agent._forecast()
print(f"预测值: {np.round(forecast, 2)}")

### 4. 可视化结果

In [ ]:
plt.figure(figsize=(15, 7))

# 绘制用于训练的历史数据
plt.plot(range(len(training_data)), training_data, label='训练数据', color='blue')

# 绘制真实的未来数据以供对比
output_window = agent_cfg["output_window_size"]
true_future_range = range(len(training_data), len(training_data) + output_window)
true_future_values = data[len(training_data) : len(training_data) + output_window]
plt.plot(true_future_range, true_future_values, 'go-', label='真实未来值')

# 绘制模型的预测值
plt.plot(true_future_range, forecast, 'ro--', label='LSTM 预测值')

plt.title("LSTM 时间序列预测", fontsize=16)
plt.xlabel("时间步")
plt.ylabel("数值")
plt.legend()
plt.grid(True)
plt.show()